# ECG arrhythmia classification using CNN

# Importing Neccessary Libraries

In [2]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

# Image Data Agumentation

In [3]:
#setting parameter for Image Data agumentation to the traing data
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [4]:
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

# Loading our data and performing data agumentation

In [5]:
#performing data agumentation to train data
x_train=train_datagen.flow_from_directory(directory=r'D:\data\train'
                                          ,target_size=(64,64),batch_size=32,class_mode='categorical')
#performing data agumentation to test data
x_test=test_datagen.flow_from_directory(directory=r'D:\data\test'
                                        ,target_size=(64,64),batch_size=32,class_mode='categorical')

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


In [6]:
print(x_train.class_indices)#checking the number of classes

{'Left Bundle Branch Block': 0, 'Normal': 1, 'Premature Atrial Contraction': 2, 'Premature Ventricular Contractions': 3, 'Right Bundle Branch Block': 4, 'Ventricular Fibrillation': 5}


In [7]:
from collections import Counter as c
c(x_train.labels)

Counter({0: 504, 1: 7346, 2: 2054, 3: 2759, 4: 2239, 5: 439})

# Creating the model

In [8]:
# create model
model=Sequential()
# adding model layer
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))#convolutional layer
model.add(MaxPooling2D(pool_size=(2,2))) #MaxPooling2D-for downsampling the input

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())#flatten the dimension of the image
model.add(Dense(32))#deeply connected neural network layers.
model.add(Dense(6,activation='softmax'))#output layer with 6 neurons

In [9]:
model.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                200736    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

# Compiling the model

In [10]:
# Compile model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# Fitting the model

In [11]:
# Fit the model
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train),
                    epochs=10, validation_data=x_test,validation_steps = len(x_test))

D:\chandra\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
480/480 [==============================] - 510s 1s/step - loss: 0.9150 - accuracy: 0.6789 - val_loss: 0.5056 - val_accuracy: 0.8356
Epoch 2/10
480/480 [==============================] - 104s 216ms/step - loss: 0.2673 - accuracy: 0.9228 - val_loss: 0.4708 - val_accuracy: 0.8547
Epoch 3/10
480/480 [==============================] - 98s 204ms/step - loss: 0.2186 - accuracy: 0.9368 - val_loss: 0.4148 - val_accuracy: 0.8719
Epoch 4/10
480/480 [==============================] - 104s 217ms/step - loss: 0.1931 - accuracy: 0.9436 - val_loss: 0.3024 - val_accuracy: 0.9027
Epoch 5/10
480/480 [==============================] - 108s 224ms/step - loss: 0.1652 - accuracy: 0.9496 - val_loss: 0.3122 - val_accuracy: 0.9084
Epoch 6/10
480/480 [==============================] - 97s 202ms/step - loss: 0.1470 - accuracy: 0.9548 - val_loss: 0.3589 - val_accuracy: 0.9026
Epoch 7/10
480/480 [==============================] - 103s 214ms/step - loss: 0.1343 - accuracy: 0.9591 - val_loss: 0.5346 - val_

In [12]:
#model.fit_generator(x_train,epochs=10,validation_data=x_test)

# Saving our model

In [13]:
# Save the model
model.save('ECG.h5')

# Predicting our results

In [14]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("ECG.h5") #loading the model for testing

In [16]:
img = image.load_img("uploads/PAC.png",target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict_classes(x)#predicting the classes
pred

D:\chandra\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([2], dtype=int64)

In [17]:
index=['Left Bundle Branch Block','Normal','Premature Atrial Contraction',
       'Premature Ventricular Contractions', 'Right Bundle Branch Block','Ventricular Fibrillation']
result=str(index[pred[0]])
result

'Premature Atrial Contraction'